In [ ]:
! pip install -r ../../requirements.txt

In [ ]:
import dotenv

dotenv.load_dotenv('.env')

In [ ]:
from langchain_community.document_loaders import WikipediaLoader

docs = WikipediaLoader(query="Artificial intelligence", load_max_docs=2, doc_content_chars_max=10000).load()
docs

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=100)
documents=text_splitter.split_documents(docs)
for doc in documents[:5]:
    print(doc.page_content, "\n")

In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(
    documents,
    embedding=OpenAIEmbeddings(),
)

In [ ]:
for similar_doc in vectorstore.similarity_search_with_score("What is AI ?", k=3):
    print(similar_doc[0].page_content, "\n")

In [ ]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

retriever.batch(["what is AI?", "who invented AI ?"])

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

parser = StrOutputParser()

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm | parser

In [ ]:
rag_chain.invoke("what is AI ?")

In [ ]:
from langchain_core.globals import set_verbose, set_debug
set_verbose(True)
set_debug(True)
rag_chain.invoke("what is AI ?")